# Unstructured & Quantitative example:
## How many right footed goals were scored in a Football match?

[Women's Euros 2022 - England vs Norway](https://www.youtube.com/watch?v=EAdTGBRSCEQ)

### Task 1 : 
1. Start the timer by hitting "Run all"
2. Watch the highlights (not the whole match) [Women's Euros 2022 - England vs Norway](https://www.youtube.com/watch?v=EAdTGBRSCEQ)
3. Note down the body part used to score each goal
4. Input "Number of 'Right Footed Goals'" below then click "Submit Query" / press "Enter"
5. That stops the timer

In [ ]:
import time

# Start the timer
task_1_start_time = time.time()

# Get an input : How many Right Footed Goals were scored?
right_footed_goals = await input("Number of 'Right Footed Goals'")

## How long did that take?

In [ ]:
time_taken_in_seconds = time.time() - task_1_start_time 
clips_length_in_seconds = 97
match_length_in_seconds = 95.0 * 60

print("Right Footed Goals in the Norway game", right_footed_goals)
print("Time Taken In Seconds", int(time_taken_in_seconds))
print("It would take at least ", int(((time_taken_in_seconds / clips_length_in_seconds ) * match_length_in_seconds) / 60), " minutes to do this for the whole match")

## What is Time?

### Money

This data is from [StatsBomb](https://statsbomb.com/), a commercial company that sells data ( this is a free sam0le to get people interested ).

## Getting the Machine to do the work

A company called StatsBomb have put in the time to look at all these matches and record a vast amount of structured data about it.

https://statsbomb.com/articles/soccer/statsbomb-release-free-360-data-womens-euro-2022-available-now/

NB. Due to running this inside teams this next block is large and wouldn't usually be needed.


In [ ]:
import micropip
await micropip.install('requests')
from js import fetch

from collections import defaultdict
import requests as req
from typing import Union
import pandas as pd
import json

EURO22MATCHES = {
 'Austria vs Northern Ireland': 3835328,
 'Austria vs Norway': 3835336,
 'Belgium vs Iceland': 3835326,
 'Denmark vs Finland': 3835329,
 'Denmark vs Spain': 3835337,
 'England vs Austria': 3835319,
 'England vs Germany': 3847567,
 'England vs Norway': 3835327,
 'England vs Spain': 3844384,
 'England vs Sweden': 3845506,
 'Finland vs Germany': 3835338,
 'France vs Belgium': 3835333,
 'France vs Italy': 3835325,
 'France vs Netherlands': 3844387,
 'Germany vs Austria': 3844385,
 'Germany vs Denmark': 3835322,
 'Germany vs France': 3845507,
 'Germany vs Spain': 3835330,
 'Iceland vs France': 3835342,
 'Italy vs Belgium': 3835341,
 'Italy vs Iceland': 3835334,
 'Netherlands vs Portugal': 3835332,
 'Netherlands vs Sweden': 3835324,
 'Northern Ireland vs England': 3835335,
 'Norway vs Northern Ireland': 3835320,
 'Portugal vs Switzerland': 3835323,
 'Spain vs Finland': 3835321,
 'Sweden vs Belgium': 3844386,
 'Sweden vs Portugal': 3835340,
 'Sweden vs Switzerland': 3835331,
 'Switzerland vs Netherlands': 3835339}

PLURALS = {
    "Starting XI": "starting_xis",
    "Half Start": "half_starts",
    "Camera On": "camera ons",
    "Camera off": "camera offs",
    "Pass": "passes",
    "Ball Receipt*": "ball_receipts",
    "Carry": "carrys",
    "Pressure": "pressures",
    "Foul Committed": "foul_committeds",
    "Foul Won": "foul_wons",
    "Duel": "duels",
    "Interception": "interceptions",
    "Block": "blocks",
    "Referee Ball-Drop": "referee_ball_drops",
    "Ball Recovery": "ball_recoverys",
    "Dispossessed": "dispossesseds",
    "Clearance": "clearances",
    "Dribble": "dribbles",
    "Miscontrol": "miscontrols",
    "Shot": "shots",
    "Goal Keeper": "goal_keepers",
    "Dribbled Past": "dribbled_pasts",
    "Injury Stoppage": "injury_stoppages",
    "Half End": "half_ends",
    "Substitution": "substitutions",
    "Shield": "shields",
    "Tactical Shift": "tactical_shifts",
    "Own Goal Against": "own_goal_againsts",
    "Own Goal For": "own_goal_fors",
    "Bad Behaviour": "bad_behaviours",
    "Player Off": "player_offs",
    "Player On": "player_ons",
    "50/50": "50/50s",
    "Error": "errors",
    "Offside": "offsides",
}


def flatten_event(event, flatten_attrs):
    if flatten_attrs:
        ev_type = event["type"]["name"].lower().replace(" ", "_").replace("*", "")
        ev_type = ev_type if event["type"]["name"] != "Goal Keeper" else "goalkeeper"
        if ev_type in event:
            for k, v in event[ev_type].items():
                event[f"{ev_type}_{k}"] = v
            del event[ev_type]

    for k, v in event.copy().items():
        if isinstance(v, dict) and "name" in v:
            event[k] = v["name"]
            if k in ["possession_team", "player", "team", "pass_recipient"]:
                event[f"{k}_id"] = v["id"]
    return event


def filter_and_group_events(events, filters, fmt, flatten_attrs):
    events_ = defaultdict(list)
    for ev in events.values():
        ev_type = PLURALS[ev["type"]["name"]]
        if not is_relevant(ev, filters):
            continue
        if fmt == "dataframe":
            ev = flatten_event(ev, flatten_attrs)
        events_[ev_type].append(ev)
    return events_


def is_relevant(event, filters):
    return all(event.get("type", {}).get("name") == v for k, v in filters.items())


OPEN_DATA_PATHS = {
    "competitions": "https://raw.githubusercontent.com/statsbomb/open-data/master/data/competitions.json",
    "matches": "https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/{competition_id}/{season_id}.json",
    "lineups": "https://raw.githubusercontent.com/statsbomb/open-data/master/data/lineups/{match_id}.json",
    "events": "https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{match_id}.json",
    "frames": "https://raw.githubusercontent.com/statsbomb/open-data/master/data/three-sixty/{match_id}.json",
}

def ents_events(events: list, match_id: int) -> dict:
    events_ = {}
    for ev in events:
        ev["match_id"] = match_id
        events_[ev["id"]] = ev
    return events_

async def get_response(path):
    '''
    response = req.get(path)
    response.raise_for_status()
    data = response.json()
    '''

    # example retrieving the default html file on the demo website
    res = await fetch(path)
    text = await res.text()
    data = json.loads(text)
    return data

async def public_events(match_id: int) -> dict:
    path = OPEN_DATA_PATHS["events"].format(match_id=match_id)
    events = await get_response(path)
    events2 = ents_events(events, match_id)
    return events2


async def sb_events(
    match_id: int,
    split: bool = False,
    filters: dict = {},
    fmt: str = "dataframe",
    flatten_attrs: bool = True,
) -> Union[pd.DataFrame, dict]:

    events = await public_events(match_id)

    if fmt == "dataframe":
        events = filter_and_group_events(events, filters, fmt, flatten_attrs)
        for ev_type, evs in events.items():
            events[ev_type] = pd.DataFrame(evs)
        if split is False:
            events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
    return events


## How many Right Footed goals in the England vs Norway game?

In [ ]:
# Start the timer
example_1_start_time = time.time() 

match_id = EURO22MATCHES['England vs Norway']

events = await sb_events(match_id=match_id)

events = events.query("type == 'Shot' and shot_outcome=='Goal'")

#Enter your answers here

right_footed_goals = len(events.query("shot_body_part == 'Right Foot'"))
time_taken_in_seconds = time.time() - example_1_start_time

print("Right Footed Goals in the Norway game from data =", right_footed_goals)
print("Time Taken In Seconds", time_taken_in_seconds)

events[['timestamp','period','shot_body_part', 'shot_outcome']]

### Task 2 : How many Headed goals in the England vs Sweden game?

Change the code below so that :
1. It's using the data for the "England vs Sweden" game. 
2. It's filtering for "Head" goals instead of "Right Foot" 
3. Click the run triangle or press "Shift + Enter"

In [ ]:
# Start the timer
task_2_start_time = time.time() 

match_id = EURO22MATCHES['England vs Norway']

events = await sb_events(match_id=match_id)

events = events.query("type == 'Shot' and shot_outcome=='Goal' and shot_body_part == 'Right Foot'")

time_taken_in_seconds = time.time() - task_2_start_time

print("Headed Goals in the Sweden game =", len(events))
print("Time Taken In Seconds", time_taken_in_seconds)

events[['timestamp','period','shot_body_part', 'shot_outcome']]


### Extension

List all the shots in the "Netherlands vs Sweden" game with the player, team and shot technique as well

The available field names are :

'50_50', 'ball_receipt_outcome', 'ball_recovery_offensive',
       'ball_recovery_recovery_failure', 'block_deflection',
       'carry_end_location', 'clearance_aerial_won', 'clearance_body_part',
       'clearance_head', 'clearance_left_foot', 'clearance_other',
       'clearance_right_foot', 'counterpress', 'dribble_no_touch',
       'dribble_nutmeg', 'dribble_outcome', 'dribble_overrun', 'duel_outcome',
       'duel_type', 'duration', 'foul_committed_advantage',
       'foul_committed_card', 'foul_committed_penalty', 'foul_won_advantage',
       'foul_won_defensive', 'foul_won_penalty', 'goalkeeper_body_part',
       'goalkeeper_end_location', 'goalkeeper_outcome', 'goalkeeper_position',
       'goalkeeper_shot_saved_off_target', 'goalkeeper_technique',
       'goalkeeper_type', 'id', 'index', 'injury_stoppage_in_chain',
       'interception_outcome', 'location', 'match_id', 'minute',
       'miscontrol_aerial_won', 'off_camera', 'out', 'pass_aerial_won',
       'pass_angle', 'pass_assisted_shot_id', 'pass_body_part', 'pass_cross',
       'pass_cut_back', 'pass_end_location', 'pass_goal_assist', 'pass_height',
       'pass_inswinging', 'pass_length', 'pass_no_touch', 'pass_outcome',
       'pass_recipient', 'pass_recipient_id', 'pass_shot_assist',
       'pass_switch', 'pass_technique', 'pass_through_ball', 'pass_type',
       'period', 'play_pattern', 'player', 'player_id', 'position',
       'possession', 'possession_team', 'possession_team_id', 'related_events',
       'second', 'shot_aerial_won', 'shot_body_part', 'shot_deflected',
       'shot_end_location', 'shot_first_time', 'shot_freeze_frame',
       'shot_key_pass_id', 'shot_open_goal', 'shot_outcome',
       'shot_saved_off_target', 'shot_statsbomb_xg', 'shot_technique',
       'shot_type', 'substitution_outcome', 'substitution_replacement',
       'tactics', 'team', 'team_id', 'timestamp', 'type', 'under_pressure'


In [ ]:
# Start the timer
match_id = EURO22MATCHES['England vs Norway']

events = await sb_events(match_id=match_id)
events = events.query("type == 'Shot' and shot_outcome=='Goal' and shot_body_part == 'Right Foot'")

events[['timestamp','period','shot_body_part', 'shot_outcome']]

# Save and Close this file